In [1]:
import pandas as pd
import numpy as np
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import cosine_similarity
from sklearn.metrics.pairwise import linear_kernel

In [2]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import random
import yaml

In [3]:
from nltk.corpus import stopwords
from scipy.sparse import csr_matrix, vstack
from scipy.sparse.linalg import svds
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import cosine_similarity
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import MinMaxScaler, normalize
from sklearn.utils import shuffle

import spotipy 
from spotipy.oauth2 import SpotifyOAuth

from tqdm import tqdm

from surprise.model_selection import cross_validate
from sklearn.model_selection import train_test_split

from recommenders.datasets.python_splitters import python_random_split

from math import sqrt

from sklearn.metrics import mean_absolute_error
from sklearn.metrics import mean_squared_error
from sklearn.metrics import ndcg_score, average_precision_score

from surprise.model_selection import KFold
from sklearn.model_selection import KFold, StratifiedKFold

from sklearn.metrics import average_precision_score
from sklearn.metrics import recall_score
from typing import List

from math import sqrt
import math

import scipy.sparse as sp
from sklearn.metrics import recall_score, precision_score


In [4]:
#POTREBNO IZMJENIT PUTANJU!!!!
#učitaj generirane podatke iz dataseta
playlist_tracks_df_1 = pd.read_pickle(r"C:\Users\Ivan\PycharmProjects\sars4\sars4\spotify\playlist_tracks.pkl")
playlist_tracks_df_2 = pd.read_pickle(r"C:\Users\Ivan\PycharmProjects\sars8\spotify\playlist_tracks_2.pkl")
playlist_tracks_df_3 = pd.read_pickle(r"C:\Users\Ivan\PycharmProjects\sars10\spotify\playlist_tracks_3.pkl")
playlist_tracks_df = pd.concat([playlist_tracks_df_1,playlist_tracks_df_2, playlist_tracks_df_3], ignore_index=True)

In [6]:
playlist_tracks_df['isin']=0

In [7]:
user_set = playlist_tracks_df['playlist_id'].unique()                           #Napravi skup jedinstvenih pjesama 
playlist_tracks_df['genres_str'] = playlist_tracks_df['genres'].apply(lambda x: ' '.join(x))   

#SPOJI SVE RIJECI KOJE SE SPOMINJU, STVORI TFIDFVECTORIZER I NAUCI NA MODELU

str_pl_total=(playlist_tracks_df['name'] + ' ' + playlist_tracks_df['artist_name'] + ' ' +playlist_tracks_df['album_name'] + ' ' +playlist_tracks_df['playlist_name'] + ' ' +playlist_tracks_df['genres_str'])
unique_str_pl_interacted = list(set(str_pl_total))
tfidf = TfidfVectorizer(analyzer='word',
                                 ngram_range=(1,2),
                                 min_df=0.003,
                                 max_df=0.5,
                                 max_features=5000,
                                 stop_words=stopwords.words('english'))
tfidf_matrix = tfidf.fit_transform(unique_str_pl_interacted)

In [32]:
#STVORI DICTIONARY U KOJEM SE NALAZE DATAFRAMEOVI GDJE CE S JEDAN BIT OZNACENE PJESME KOJE JE POJEDINI KORISNIK SLUSAO

playlist_tracks_df_dict={}
for user in tqdm(user_set):
    copy_playlist_tracks_df=playlist_tracks_df.copy()
    copy_playlist_tracks_df.loc[copy_playlist_tracks_df['playlist_id'] == user, 'isin'] = 1
    playlist_tracks_df_dict[user]=copy_playlist_tracks_df

100%|███████████████████████████████████████████████████████████████████████████████| 149/149 [00:01<00:00, 137.75it/s]


In [9]:
def get_interacted_tracks(tracks, user, drop_duplicates=True):
    interacted_track_id_df=pd.DataFrame()
    for row in tracks.index:
        if(tracks['playlist_id'][row]==user):
            interacted_track_id_df=interacted_track_id_df.append(tracks.iloc[row], ignore_index=True)
            
    interacted_track_id_df = interacted_track_id_df.drop_duplicates(subset='id', keep="first").reset_index()
    return interacted_track_id_df


In [10]:
#RADE SE PREPORUKE PREKO SLICNOSTI KOSINUSA

def get_recommendations(p_df, df,tps_norm, tfidf_matrix, interacted):
    cosine_similarity_scores=linear_kernel(tps_norm, tfidf_matrix)
    similar_indices=cosine_similarity_scores.argsort()[0]

    s=pd.DataFrame({"index":[],"name":[], "id":[], "pred":[], "isin":[]})
    
    for i in similar_indices:
        df_i={"index":p_df.index.tolist()[i], "name":p_df['name'].tolist()[i],
              "id":p_df['id'].tolist()[i],
              "pred":cosine_similarity_scores[0,i], "isin": p_df['isin'].tolist()[i]}
        s=s.append(df_i, ignore_index=True)
    s.sort_values(by=['pred'], ascending=False)
    
    interacted_indexes_set=set()
    for elem in interacted['id']:
        interacted_indexes=(s[s['id']==elem].index.values)
        for el in interacted_indexes:
            interacted_indexes_set.add(int(el))
    
    df1=pd.DataFrame({"index":[],"name":[], "id":[], "pred":[], "isin":[]})
    df1=s.drop(interacted_indexes_set)
    df_reverse_rows = df1.iloc[::-1]
    return df_reverse_rows

In [11]:
#PRVO SE ZA SVAKOG KORISNIKA DOBIJU PJESME S KOJIMA JE IMAO INTERAKCIJE, ONDA SE IZVUKU RIJECI IZ PJESAMA, ONDA SE IZBACUJU DUPLICI
#POTOM SE PJESME DODAJU U TFIDF MATRICU, NAKON CEGA SE OBAVLJA PREPORUCIVANJE

interacted_dict={}
content_model_recommendations_dict={}
for user in tqdm(user_set):
    interacted=get_interacted_tracks(playlist_tracks_df_dict[user], user)
    interacted_dict[user]=interacted.copy()
    
    str_pl_interacted=(interacted['name'] + ' ' + interacted['artist_name'] 
                       + ' ' +interacted['album_name'] + ' ' +interacted['playlist_name'] +
                       ' ' +interacted['genres_str'])
    unique_str_pl_interacted = list(set(str_pl_interacted))    
    
    tps=[]
    for elem in interacted['id']: 
        idx = playlist_tracks_df['id'].tolist().index(elem)
        track_profile = tfidf_matrix[idx:idx+1]
        tps.append(track_profile)

    tps_stack = vstack(tps)
    tps_array = np.sum(tps_stack, axis=0)  
    tps_norm = normalize(tps_array)
    content_model_recommendations_dict[user]=get_recommendations(playlist_tracks_df_dict[user],
                                                                 unique_str_pl_interacted,
                                                                 tps_norm, tfidf_matrix, interacted)


  0%|                                                                                          | 0/149 [00:00<?, ?it/s]c:\users\ivan\appdata\local\programs\python\python37\lib\site-packages\sklearn\utils\validation.py:598: FutureWarning: np.matrix usage is deprecated in 1.0 and will raise a TypeError in 1.2. Please convert to a numpy array with np.asarray. For more information see: https://numpy.org/doc/stable/reference/generated/numpy.matrix.html
  FutureWarning,
  1%|▌                                                                              | 1/149 [01:52<4:37:26, 112.48s/it]c:\users\ivan\appdata\local\programs\python\python37\lib\site-packages\sklearn\utils\validation.py:598: FutureWarning: np.matrix usage is deprecated in 1.0 and will raise a TypeError in 1.2. Please convert to a numpy array with np.asarray. For more information see: https://numpy.org/doc/stable/reference/generated/numpy.matrix.html
  FutureWarning,
  1%|█                                                        

 12%|█████████▌                                                                     | 18/149 [32:24<3:36:36, 99.21s/it]c:\users\ivan\appdata\local\programs\python\python37\lib\site-packages\sklearn\utils\validation.py:598: FutureWarning: np.matrix usage is deprecated in 1.0 and will raise a TypeError in 1.2. Please convert to a numpy array with np.asarray. For more information see: https://numpy.org/doc/stable/reference/generated/numpy.matrix.html
  FutureWarning,
 13%|██████████                                                                     | 19/149 [33:55<3:30:00, 96.93s/it]c:\users\ivan\appdata\local\programs\python\python37\lib\site-packages\sklearn\utils\validation.py:598: FutureWarning: np.matrix usage is deprecated in 1.0 and will raise a TypeError in 1.2. Please convert to a numpy array with np.asarray. For more information see: https://numpy.org/doc/stable/reference/generated/numpy.matrix.html
  FutureWarning,
 13%|██████████▌                                              

 24%|███████████████████                                                            | 36/149 [50:39<1:35:40, 50.80s/it]c:\users\ivan\appdata\local\programs\python\python37\lib\site-packages\sklearn\utils\validation.py:598: FutureWarning: np.matrix usage is deprecated in 1.0 and will raise a TypeError in 1.2. Please convert to a numpy array with np.asarray. For more information see: https://numpy.org/doc/stable/reference/generated/numpy.matrix.html
  FutureWarning,
 25%|███████████████████▌                                                           | 37/149 [51:29<1:34:16, 50.50s/it]c:\users\ivan\appdata\local\programs\python\python37\lib\site-packages\sklearn\utils\validation.py:598: FutureWarning: np.matrix usage is deprecated in 1.0 and will raise a TypeError in 1.2. Please convert to a numpy array with np.asarray. For more information see: https://numpy.org/doc/stable/reference/generated/numpy.matrix.html
  FutureWarning,
 26%|████████████████████▏                                    

 36%|███████████████████████████▉                                                 | 54/149 [1:06:03<1:21:16, 51.33s/it]c:\users\ivan\appdata\local\programs\python\python37\lib\site-packages\sklearn\utils\validation.py:598: FutureWarning: np.matrix usage is deprecated in 1.0 and will raise a TypeError in 1.2. Please convert to a numpy array with np.asarray. For more information see: https://numpy.org/doc/stable/reference/generated/numpy.matrix.html
  FutureWarning,
 37%|████████████████████████████▍                                                | 55/149 [1:06:50<1:18:24, 50.05s/it]c:\users\ivan\appdata\local\programs\python\python37\lib\site-packages\sklearn\utils\validation.py:598: FutureWarning: np.matrix usage is deprecated in 1.0 and will raise a TypeError in 1.2. Please convert to a numpy array with np.asarray. For more information see: https://numpy.org/doc/stable/reference/generated/numpy.matrix.html
  FutureWarning,
 38%|████████████████████████████▉                            

 48%|██████████████████████████████████████▏                                        | 72/149 [1:19:51<55:12, 43.02s/it]c:\users\ivan\appdata\local\programs\python\python37\lib\site-packages\sklearn\utils\validation.py:598: FutureWarning: np.matrix usage is deprecated in 1.0 and will raise a TypeError in 1.2. Please convert to a numpy array with np.asarray. For more information see: https://numpy.org/doc/stable/reference/generated/numpy.matrix.html
  FutureWarning,
 49%|██████████████████████████████████████▋                                        | 73/149 [1:20:33<54:04, 42.70s/it]c:\users\ivan\appdata\local\programs\python\python37\lib\site-packages\sklearn\utils\validation.py:598: FutureWarning: np.matrix usage is deprecated in 1.0 and will raise a TypeError in 1.2. Please convert to a numpy array with np.asarray. For more information see: https://numpy.org/doc/stable/reference/generated/numpy.matrix.html
  FutureWarning,
 50%|███████████████████████████████████████▏                 

 60%|███████████████████████████████████████████████▋                               | 90/149 [1:32:33<41:39, 42.37s/it]c:\users\ivan\appdata\local\programs\python\python37\lib\site-packages\sklearn\utils\validation.py:598: FutureWarning: np.matrix usage is deprecated in 1.0 and will raise a TypeError in 1.2. Please convert to a numpy array with np.asarray. For more information see: https://numpy.org/doc/stable/reference/generated/numpy.matrix.html
  FutureWarning,
 61%|████████████████████████████████████████████████▏                              | 91/149 [1:33:16<41:08, 42.56s/it]c:\users\ivan\appdata\local\programs\python\python37\lib\site-packages\sklearn\utils\validation.py:598: FutureWarning: np.matrix usage is deprecated in 1.0 and will raise a TypeError in 1.2. Please convert to a numpy array with np.asarray. For more information see: https://numpy.org/doc/stable/reference/generated/numpy.matrix.html
  FutureWarning,
 62%|████████████████████████████████████████████████▊        

 72%|████████████████████████████████████████████████████████▌                     | 108/149 [1:45:20<28:58, 42.40s/it]c:\users\ivan\appdata\local\programs\python\python37\lib\site-packages\sklearn\utils\validation.py:598: FutureWarning: np.matrix usage is deprecated in 1.0 and will raise a TypeError in 1.2. Please convert to a numpy array with np.asarray. For more information see: https://numpy.org/doc/stable/reference/generated/numpy.matrix.html
  FutureWarning,
 73%|█████████████████████████████████████████████████████████                     | 109/149 [1:46:03<28:26, 42.67s/it]c:\users\ivan\appdata\local\programs\python\python37\lib\site-packages\sklearn\utils\validation.py:598: FutureWarning: np.matrix usage is deprecated in 1.0 and will raise a TypeError in 1.2. Please convert to a numpy array with np.asarray. For more information see: https://numpy.org/doc/stable/reference/generated/numpy.matrix.html
  FutureWarning,
 74%|█████████████████████████████████████████████████████████

 85%|█████████████████████████████████████████████████████████████████▉            | 126/149 [1:58:10<16:16, 42.47s/it]c:\users\ivan\appdata\local\programs\python\python37\lib\site-packages\sklearn\utils\validation.py:598: FutureWarning: np.matrix usage is deprecated in 1.0 and will raise a TypeError in 1.2. Please convert to a numpy array with np.asarray. For more information see: https://numpy.org/doc/stable/reference/generated/numpy.matrix.html
  FutureWarning,
 85%|██████████████████████████████████████████████████████████████████▍           | 127/149 [1:58:51<15:25, 42.07s/it]c:\users\ivan\appdata\local\programs\python\python37\lib\site-packages\sklearn\utils\validation.py:598: FutureWarning: np.matrix usage is deprecated in 1.0 and will raise a TypeError in 1.2. Please convert to a numpy array with np.asarray. For more information see: https://numpy.org/doc/stable/reference/generated/numpy.matrix.html
  FutureWarning,
 86%|█████████████████████████████████████████████████████████

 97%|███████████████████████████████████████████████████████████████████████████▍  | 144/149 [2:10:57<03:56, 47.21s/it]c:\users\ivan\appdata\local\programs\python\python37\lib\site-packages\sklearn\utils\validation.py:598: FutureWarning: np.matrix usage is deprecated in 1.0 and will raise a TypeError in 1.2. Please convert to a numpy array with np.asarray. For more information see: https://numpy.org/doc/stable/reference/generated/numpy.matrix.html
  FutureWarning,
 97%|███████████████████████████████████████████████████████████████████████████▉  | 145/149 [2:11:45<03:09, 47.40s/it]c:\users\ivan\appdata\local\programs\python\python37\lib\site-packages\sklearn\utils\validation.py:598: FutureWarning: np.matrix usage is deprecated in 1.0 and will raise a TypeError in 1.2. Please convert to a numpy array with np.asarray. For more information see: https://numpy.org/doc/stable/reference/generated/numpy.matrix.html
  FutureWarning,
 98%|█████████████████████████████████████████████████████████

In [12]:
#Izračunaj koliko se koja pjesma ponavlja u preporukama za sve korisnike
preds_dict_rec_100={}
items_count_dict={}
for i in playlist_tracks_df.index:
    items_count_dict[i]=0

for playlist in tqdm(playlist_tracks_df_dict):
    preds_dict_rec_100[playlist]=content_model_recommendations_dict[playlist].head(99)
    for item in preds_dict_rec_100[playlist].index:
        items_count_dict[item]+=1

100%|██████████████████████████████████████████████████████████████████████████████| 149/149 [00:00<00:00, 4140.61it/s]


In [13]:
#PROSJECNI STOTI ELEMENT

average_100th_list=[]
for user in tqdm(content_model_recommendations_dict):
    average_100th_list.append(content_model_recommendations_dict[user]['pred'].sort_values(ascending=False).iloc[99])
average_100th=sum(average_100th_list)/len(average_100th_list)
print("AVERAGE 100TH:  ",average_100th)

100%|███████████████████████████████████████████████████████████████████████████████| 149/149 [00:00<00:00, 628.90it/s]

AVERAGE 100TH:   0.43752368469861835


In [14]:
content_model_recommendations_dict[user].head(25)

,index,name,id,pred,isin
9998,6006.0,Break Point,6kTHDRSpUnkDmsmarHBJqK,0.460346,0.0
9995,4012.0,Miami Sky,6HMiE1sM5OYJpFXEn1FBc9,0.441895,0.0
9994,9801.0,You've Got Another Thing Coming,4f3RDq9nYPBeR1yMSgnmBm,0.432212,0.0
9993,929.0,Every Breath You Take,1JSTJqkT5qHq8MDJnJbRE1,0.425464,0.0
9992,9191.0,Walk This Way (feat. Aerosmith),6qUEOWqOzu1rLPUPQ1ECpx,0.424950,0.0
9991,9446.0,I Want To Break Free - Remastered 2011,1MsBRSbt5dqJSw3RxXtvCM,0.423997,0.0
9990,2070.0,"Tugo, Nesrećo",5Ja2Y4in46OBZfeTAvg2uv,0.423487,0.0
9989,5737.0,Hooked On A Feeling,6Ac4NVYYl2U73QiTt11ZKd,0.421478,0.0
9988,1862.0,Rainbow In The Dark,5qFxuUIAwJlzO60vqOiL7y,0.419573,0.0
9987,565.0,Livin' On A Prayer,7qQnBfwXrw2tZNFG4Uf57N,0.419189,0.0


In [15]:
print("TEST-TRAIN-TEST-TRAIN-TEST-TRAIN-TEST-TRAIN-TEST-TRAIN-TEST-TRAIN-TEST-TRAIN-TEST-TRAIN-TEST-TRAIN-TEST-TRAIN-")

TEST-TRAIN-TEST-TRAIN-TEST-TRAIN-TEST-TRAIN-TEST-TRAIN-TEST-TRAIN-TEST-TRAIN-TEST-TRAIN-TEST-TRAIN-TEST-TRAIN-


In [16]:
def get_recommendations_tt(p_df, df,tps_norm, tfidf_matrix, interacted):
    cosine_similarity_scores=linear_kernel(tps_norm, tfidf_matrix)
    similar_indices=cosine_similarity_scores.argsort()[0]

    s=pd.DataFrame({"index":[],"name":[], "id":[], "pred":[], "isin":[]})
    for i in similar_indices:
        
        df_i={"index":p_df.index.tolist()[i], "name":p_df['name'].tolist()[i],
              "id":p_df['id'].tolist()[i],
              "pred":cosine_similarity_scores[0,i], "isin": p_df['isin'].tolist()[i]}
        s=s.append(df_i, ignore_index=True)
    s.sort_values(by=['pred'], ascending=False)
    
    interacted_indexes_set=set()
    for elem in interacted['id']:
        interacted_indexes=(s[s['id']==elem].index.values)
        for el in interacted_indexes:
            interacted_indexes_set.add(int(el))
    
    df1=pd.DataFrame({"index":[],"name":[], "id":[], "pred":[], "isin":[]})
    df1=s.drop(interacted_indexes_set)
    df_reverse_rows = df1.iloc[::-1]
    return df_reverse_rows

In [17]:
def get_interacted_tracks_tt(tracks, user, drop_duplicates=True):
    interacted_track_id_df=pd.DataFrame()
    inbounds=True
    
    for row in tracks.index:
        if(tracks['playlist_id'][row]==user):
            if(0 <= row < tracks.shape[0]):
                interacted_track_id_df=interacted_track_id_df.append(tracks.iloc[row], ignore_index=True)
            else:
                inbounds=False
    interacted_track_id_df = interacted_track_id_df.drop_duplicates(subset='id', keep="first").reset_index()
    zipped_pairs = []
    zipped_pairs.append(interacted_track_id_df)
    zipped_pairs.append(inbounds)
    return zipped_pairs


In [18]:
rmse_list_tt=[]
mae_list_tt=[]
ndcg_list_tt=[]
map_list_tt=[]

precision_list_tt_micro=[]
precision_list_tt_macro=[]
precision_list_tt_weighted=[]

recall_list_tt_micro=[]
recall_list_tt_macro=[]
recall_list_tt_weighted=[]

content_model_recommendations_dict_tt={}

for u in (content_model_recommendations_dict):

    train_TT, test_TT = python_random_split(content_model_recommendations_dict[u], 0.8)
    
    #EVALUACIJA
    
    rmse_list_tt.append(sqrt(mean_squared_error(test_TT['pred'], test_TT['isin'])))
    mae_list_tt.append((mean_absolute_error(test_TT['pred'], test_TT['isin'])))
    ndcg_list_tt.append(ndcg_score(test_TT['isin'].values.reshape(1, -1), test_TT['pred'].values.reshape(1, -1)))
    
    map_score=average_precision_score(test_TT['isin'].values, test_TT['pred'].values)
    if not math.isnan(map_score):
        map_list_tt.append(map_score)
    
    
    df=test_TT.copy()
    df.loc[df['pred']>=average_100th, 'pred']=1
    df.loc[df['pred']<average_100th, 'pred']=0
    
    precision_list_tt_micro.append(precision_score(df['isin'], df['pred'], average='micro'))
    precision_list_tt_macro.append(precision_score(df['isin'], df['pred'], average='macro'))
    precision_list_tt_weighted.append(precision_score(df['isin'], df['pred'], average='weighted'))

    recall_list_tt_micro.append(recall_score(df['isin'], df['pred'], average='micro'))
    recall_list_tt_macro.append(recall_score(df['isin'], df['pred'], average='macro'))
    recall_list_tt_weighted.append(recall_score(df['isin'], df['pred'], average='weighted'))

    
rmse_mean_tt=np.mean(rmse_list_tt)
mae_mean_tt=np.mean(mae_list_tt)
ndcg_mean_tt=np.mean(ndcg_list_tt)
map_mean_tt=np.mean(map_list_tt)

precision_tt_micro=np.mean(precision_list_tt_micro)
precision_tt_macro=np.mean(precision_list_tt_macro)
precision_tt_weighted=np.mean(precision_list_tt_weighted)

recall_tt_micro=np.mean(recall_list_tt_micro)
recall_tt_macro=np.mean(recall_list_tt_macro)
recall_tt_weighted=np.mean(recall_list_tt_weighted)

f1_tt_micro=(2*precision_tt_micro*recall_tt_micro)/(precision_tt_micro+recall_tt_micro)
f1_tt_macro=(2*precision_tt_macro*recall_tt_macro)/(precision_tt_macro+recall_tt_macro)
f1_tt_weighted=(2*precision_tt_weighted*recall_tt_weighted)/(precision_tt_weighted+recall_tt_weighted)


print('TRAIN-TEST SKLEARN.METRICS RMSE:   ',rmse_mean_tt)
print('TRAIN-TEST SKLEARN.METRICS MAE:   ',mae_mean_tt)
print('TRAIN-TEST SKLEARN.METRICS MAP :   ',map_mean_tt)
print('TRAIN-TEST SKLEARN.METRICS NDCG:   ',ndcg_mean_tt)

print('TRAIN-TEST SKLEARN.METRICS PRECISION: ',precision_tt_micro ,"|" ,precision_tt_macro ,"|" ,precision_tt_weighted )
print('TRAIN-TEST SKLEARN.METRICS RECALL: ' ,recall_tt_micro ,"|" ,recall_tt_macro ,"|" ,recall_tt_weighted )
print("TRAIN-TEST SKLEARN.METRICS F1:   ", f1_tt_micro ,"|" ,f1_tt_macro ,"|" ,f1_tt_weighted )

c:\users\ivan\appdata\local\programs\python\python37\lib\site-packages\sklearn\metrics\_ranking.py:864: RuntimeWarning: invalid value encountered in true_divide
  recall = tps / tps[-1]
c:\users\ivan\appdata\local\programs\python\python37\lib\site-packages\sklearn\metrics\_classification.py:1318: UndefinedMetricWarning: Recall is ill-defined and being set to 0.0 in labels with no true samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
c:\users\ivan\appdata\local\programs\python\python37\lib\site-packages\sklearn\metrics\_classification.py:1318: UndefinedMetricWarning: Recall is ill-defined and being set to 0.0 in labels with no true samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
c:\users\ivan\appdata\local\programs\python\python37\lib\site-packages\sklearn\metrics\_ranking.py:864: RuntimeWarning: invalid value encountered in true_divide
  recall 

c:\users\ivan\appdata\local\programs\python\python37\lib\site-packages\sklearn\metrics\_classification.py:1318: UndefinedMetricWarning: Recall is ill-defined and being set to 0.0 in labels with no true samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
c:\users\ivan\appdata\local\programs\python\python37\lib\site-packages\sklearn\metrics\_classification.py:1318: UndefinedMetricWarning: Recall is ill-defined and being set to 0.0 in labels with no true samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
c:\users\ivan\appdata\local\programs\python\python37\lib\site-packages\sklearn\metrics\_ranking.py:864: RuntimeWarning: invalid value encountered in true_divide
  recall = tps / tps[-1]
c:\users\ivan\appdata\local\programs\python\python37\lib\site-packages\sklearn\metrics\_classification.py:1318: UndefinedMetricWarning: Recall is ill-defined and being se

c:\users\ivan\appdata\local\programs\python\python37\lib\site-packages\sklearn\metrics\_classification.py:1318: UndefinedMetricWarning: Recall is ill-defined and being set to 0.0 in labels with no true samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
c:\users\ivan\appdata\local\programs\python\python37\lib\site-packages\sklearn\metrics\_classification.py:1318: UndefinedMetricWarning: Recall is ill-defined and being set to 0.0 in labels with no true samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
c:\users\ivan\appdata\local\programs\python\python37\lib\site-packages\sklearn\metrics\_ranking.py:864: RuntimeWarning: invalid value encountered in true_divide
  recall = tps / tps[-1]
c:\users\ivan\appdata\local\programs\python\python37\lib\site-packages\sklearn\metrics\_classification.py:1318: UndefinedMetricWarning: Recall is ill-defined and being se

c:\users\ivan\appdata\local\programs\python\python37\lib\site-packages\sklearn\metrics\_classification.py:1318: UndefinedMetricWarning: Recall is ill-defined and being set to 0.0 in labels with no true samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
c:\users\ivan\appdata\local\programs\python\python37\lib\site-packages\sklearn\metrics\_classification.py:1318: UndefinedMetricWarning: Recall is ill-defined and being set to 0.0 in labels with no true samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
c:\users\ivan\appdata\local\programs\python\python37\lib\site-packages\sklearn\metrics\_ranking.py:864: RuntimeWarning: invalid value encountered in true_divide
  recall = tps / tps[-1]
c:\users\ivan\appdata\local\programs\python\python37\lib\site-packages\sklearn\metrics\_classification.py:1318: UndefinedMetricWarning: Recall is ill-defined and being se

c:\users\ivan\appdata\local\programs\python\python37\lib\site-packages\sklearn\metrics\_ranking.py:864: RuntimeWarning: invalid value encountered in true_divide
  recall = tps / tps[-1]
c:\users\ivan\appdata\local\programs\python\python37\lib\site-packages\sklearn\metrics\_classification.py:1318: UndefinedMetricWarning: Recall is ill-defined and being set to 0.0 in labels with no true samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
c:\users\ivan\appdata\local\programs\python\python37\lib\site-packages\sklearn\metrics\_classification.py:1318: UndefinedMetricWarning: Recall is ill-defined and being set to 0.0 in labels with no true samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
c:\users\ivan\appdata\local\programs\python\python37\lib\site-packages\sklearn\metrics\_ranking.py:864: RuntimeWarning: invalid value encountered in true_divide
  recall 

...

c:\users\ivan\appdata\local\programs\python\python37\lib\site-packages\sklearn\metrics\_classification.py:1318: UndefinedMetricWarning: Recall is ill-defined and being set to 0.0 in labels with no true samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
c:\users\ivan\appdata\local\programs\python\python37\lib\site-packages\sklearn\metrics\_classification.py:1318: UndefinedMetricWarning: Recall is ill-defined and being set to 0.0 in labels with no true samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
c:\users\ivan\appdata\local\programs\python\python37\lib\site-packages\sklearn\metrics\_ranking.py:864: RuntimeWarning: invalid value encountered in true_divide
  recall = tps / tps[-1]
c:\users\ivan\appdata\local\programs\python\python37\lib\site-packages\sklearn\metrics\_classification.py:1318: UndefinedMetricWarning: Recall is ill-defined and being se

c:\users\ivan\appdata\local\programs\python\python37\lib\site-packages\sklearn\metrics\_classification.py:1318: UndefinedMetricWarning: Recall is ill-defined and being set to 0.0 in labels with no true samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
c:\users\ivan\appdata\local\programs\python\python37\lib\site-packages\sklearn\metrics\_classification.py:1318: UndefinedMetricWarning: Recall is ill-defined and being set to 0.0 in labels with no true samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
c:\users\ivan\appdata\local\programs\python\python37\lib\site-packages\sklearn\metrics\_ranking.py:864: RuntimeWarning: invalid value encountered in true_divide
  recall = tps / tps[-1]
c:\users\ivan\appdata\local\programs\python\python37\lib\site-packages\sklearn\metrics\_classification.py:1318: UndefinedMetricWarning: Recall is ill-defined and being se

c:\users\ivan\appdata\local\programs\python\python37\lib\site-packages\sklearn\metrics\_classification.py:1318: UndefinedMetricWarning: Recall is ill-defined and being set to 0.0 in labels with no true samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
c:\users\ivan\appdata\local\programs\python\python37\lib\site-packages\sklearn\metrics\_classification.py:1318: UndefinedMetricWarning: Recall is ill-defined and being set to 0.0 in labels with no true samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
c:\users\ivan\appdata\local\programs\python\python37\lib\site-packages\sklearn\metrics\_ranking.py:864: RuntimeWarning: invalid value encountered in true_divide
  recall = tps / tps[-1]
c:\users\ivan\appdata\local\programs\python\python37\lib\site-packages\sklearn\metrics\_classification.py:1318: UndefinedMetricWarning: Recall is ill-defined and being se

TRAIN-TEST SKLEARN.METRICS RMSE:    0.19528888125687763
TRAIN-TEST SKLEARN.METRICS MAE:    0.1638832637715601
TRAIN-TEST SKLEARN.METRICS MAP :    nan
TRAIN-TEST SKLEARN.METRICS NDCG:    0.0
TRAIN-TEST SKLEARN.METRICS PRECISION:  0.9820667758685764 | 0.5201342281879194 | 1.0
TRAIN-TEST SKLEARN.METRICS RECALL:  0.9820667758685764 | 0.5111676161222076 | 0.9820667758685764
TRAIN-TEST SKLEARN.METRICS F1:    0.9820667758685764 | 0.5156119422325603 | 0.9909522603628906


c:\users\ivan\appdata\local\programs\python\python37\lib\site-packages\sklearn\metrics\_classification.py:1318: UndefinedMetricWarning: Recall is ill-defined and being set to 0.0 in labels with no true samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
c:\users\ivan\appdata\local\programs\python\python37\lib\site-packages\sklearn\metrics\_classification.py:1318: UndefinedMetricWarning: Recall is ill-defined and being set to 0.0 in labels with no true samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
c:\users\ivan\appdata\local\programs\python\python37\lib\site-packages\sklearn\metrics\_ranking.py:864: RuntimeWarning: invalid value encountered in true_divide
  recall = tps / tps[-1]
c:\users\ivan\appdata\local\programs\python\python37\lib\site-packages\sklearn\metrics\_classification.py:1318: UndefinedMetricWarning: Recall is ill-defined and being se

In [19]:
print("CROSS VALIDATION-CROSS VALIDATION-CROSS VALIDATION-CROSS VALIDATION-CROSS VALIDATION-CROSS VALIDATION-CROSS VALIDATION")

CROSS VALIDATION-CROSS VALIDATION-CROSS VALIDATION-CROSS VALIDATION-CROSS VALIDATION-CROSS VALIDATION-CROSS VALIDATION


In [20]:
def get_recommendations_cv(p_df, df,tps_norm, tfidf_matrix, interacted):
        
    cosine_similarity_scores=linear_kernel(tps_norm, tfidf_matrix)
    similar_indices=cosine_similarity_scores.argsort()[0]

    s=pd.DataFrame({"index":[],"name":[], "id":[], "pred":[], "isin":[]})
    
    for i in similar_indices:
        df_i={"index":p_df.index.tolist()[i], "name":p_df['name'].tolist()[i], "id":p_df['id'].tolist()[i],
              "pred":cosine_similarity_scores[0,i], "isin": p_df['isin'].tolist()[i]}
        s=s.append(df_i, ignore_index=True)
    s.sort_values(by=['pred'], ascending=False)
    
    interacted_indexes_set=set()
    for elem in interacted['id']:
        interacted_indexes=(s[s['id']==elem].index.values)
        for el in interacted_indexes:
            interacted_indexes_set.add(int(el))
    
    df1=pd.DataFrame({"index":[],"name":[], "id":[], "pred":[], "isin":[]})
    df1=s.drop(interacted_indexes_set)
    df_reverse_rows = df1.iloc[::-1]
    return df_reverse_rows

In [21]:
def get_interacted_tracks_cv(tracks, user, drop_duplicates=True):
    interacted_track_id_df=pd.DataFrame()
    for row in tracks['index']:
        condition = (tracks['index'] == row)
        value= tracks.loc[condition, 'playlist_id'].values[0]
        if(value==user):
            interacted_track_id_df=interacted_track_id_df.append(tracks[tracks['index'] == row], ignore_index=True)

    interacted_track_id_df = interacted_track_id_df.drop_duplicates(subset='id', keep="first").reset_index()
    return interacted_track_id_df

In [22]:
n_splits = 10
kf = KFold(n_splits=n_splits, shuffle=True, random_state=42)
rmse_scores1 = {}
mae_scores1 = {}
precision_scores1 = {}
recall_scores1 = {}
map_scores1 = {}
ndcg_scores1 = {}
content_model_recommendations_dict_cv={}
playlist_keys= list(playlist_tracks_df_dict.keys())
interacted_dict_cv={}
content_model_recommendations_dict_cv={}
pl=0

playlist_rmse_scores = []
playlist_mae_scores = []
playlist_map_scores = []
playlist_ndcg_scores = []

precision_results_cv_micro=[]
precision_results_cv_macro=[]
precision_results_cv_weighted=[]
recall_results_cv_micro=[]
recall_results_cv_macro=[]
recall_results_cv_weighted=[]

skf = StratifiedKFold(n_splits=10, shuffle=True, random_state=42)
nr=1
for (train_idx, test_idx) in (skf.split(playlist_tracks_df, playlist_tracks_df['isin'])):        
    train_data = playlist_tracks_df.iloc[train_idx]
    test_data = playlist_tracks_df.iloc[test_idx]
    
    #PONAVLJANJE NA VRHU NAPRAVLJENOG PROCESA
    
    for idx_i in test_idx:
        test_data.at[idx_i, 'index']=idx_i
        
    str_pl_total_cv=(test_data['name'] + ' ' + test_data['artist_name'] + ' ' +test_data['album_name'] + ' ' +
                         test_data['playlist_name'] + ' ' +test_data['genres_str'])
    unique_str_pl_interacted_cv = list(set(str_pl_total_cv))
    tfidf_cv = TfidfVectorizer(analyzer='word',
                                         ngram_range=(1,2),
                                         min_df=0.003,
                                         max_df=0.5,
                                         max_features=5000,
                                         stop_words=stopwords.words('english'))
    tfidf_matrix_cv = tfidf_cv.fit_transform(unique_str_pl_interacted_cv)
    content_model_recommendations_dict_cv={}
    interacted_cv=get_interacted_tracks_cv(test_data, playlist)

    us=0
    
    for user in (user_set):
        print(str(nr)+"us",us)
        str_pl_interacted_cv=(interacted_cv['name'] + ' ' + interacted_cv['artist_name'] + ' ' +interacted_cv['album_name']
                                  + ' ' +interacted_cv['playlist_name'] + ' ' +interacted_cv['genres_str'])
        unique_str_pl_interacted_cv = list(set(str_pl_interacted_cv))    
        tps_cv=[]
            
        for elem in interacted_cv['id']: 
            idx = test_data['id'].tolist().index(elem)
            track_profile_cv = tfidf_matrix_cv[idx:idx+1]
            tps_cv.append(track_profile_cv)
            
        tps_stack_cv = vstack(tps_cv)
        tps_array_cv = np.sum(tps_stack_cv, axis=0)  
        tps_norm_cv = normalize(tps_array_cv)
        content_model_recommendations_dict_cv[user]=get_recommendations_cv(test_data, unique_str_pl_interacted_cv,
                                                                         tps_norm_cv, tfidf_matrix_cv, interacted_cv)
        us+=1

        
        predictions=content_model_recommendations_dict_cv[user]['pred']
        true_values=content_model_recommendations_dict_cv[user]['isin']
        
        #EVALUACIJA
        
        average_rmse_CV=sqrt(mean_squared_error(predictions, true_values))
        playlist_rmse_scores.append(average_rmse_CV)
            
        average_mae_CV=(mean_absolute_error(predictions, true_values))
        playlist_mae_scores.append(average_mae_CV)
        
        average_map = average_precision_score(true_values.values, predictions.values)
        playlist_map_scores.append(average_map)
            
        average_ndcg=ndcg_score(true_values.values.reshape(1, -1), predictions.values.reshape(1, -1))
        playlist_ndcg_scores.append(average_ndcg)
                    
        average_100th_list_cv=[]
        for user in (content_model_recommendations_dict_cv):
            average_100th_list_cv.append(content_model_recommendations_dict_cv[user]['pred'].sort_values(ascending=False).iloc[99])
        average_100th_cv=sum(average_100th_list_cv)/len(average_100th_list_cv)
        
        
        predictions2=predictions.copy()
        predictions2.loc[predictions2>=average_100th_cv]=1
        predictions2.loc[predictions2<average_100th_cv]=0
        
        precision_results_cv_micro.append(precision_score(true_values, predictions2, average='micro'))
        recall_results_cv_micro.append(recall_score(true_values, predictions2, average='micro'))

        precision_results_cv_macro.append(precision_score(true_values, predictions2, average='macro'))
        recall_results_cv_macro.append(recall_score(true_values, predictions2, average='macro'))

        precision_results_cv_weighted.append(precision_score(true_values, predictions2, average='weighted'))
        recall_results_cv_weighted.append(recall_score(true_values, predictions2, average='weighted'))
    nr+=1

c:\users\ivan\appdata\local\programs\python\python37\lib\site-packages\pandas\core\indexing.py:1684: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self.obj[key] = infer_fill_value(value)
c:\users\ivan\appdata\local\programs\python\python37\lib\site-packages\pandas\core\indexing.py:1817: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self._setitem_single_column(loc, value, pi)


1us 0


c:\users\ivan\appdata\local\programs\python\python37\lib\site-packages\sklearn\utils\validation.py:598: FutureWarning: np.matrix usage is deprecated in 1.0 and will raise a TypeError in 1.2. Please convert to a numpy array with np.asarray. For more information see: https://numpy.org/doc/stable/reference/generated/numpy.matrix.html
  FutureWarning,
c:\users\ivan\appdata\local\programs\python\python37\lib\site-packages\sklearn\metrics\_ranking.py:864: RuntimeWarning: invalid value encountered in true_divide
  recall = tps / tps[-1]
c:\users\ivan\appdata\local\programs\python\python37\lib\site-packages\sklearn\metrics\_classification.py:1318: UndefinedMetricWarning: Recall is ill-defined and being set to 0.0 in labels with no true samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
c:\users\ivan\appdata\local\programs\python\python37\lib\site-packages\sklearn\metrics\_classification.py:1318: UndefinedMetricWarning: Recall

1us 1


c:\users\ivan\appdata\local\programs\python\python37\lib\site-packages\sklearn\metrics\_ranking.py:864: RuntimeWarning: invalid value encountered in true_divide
  recall = tps / tps[-1]
c:\users\ivan\appdata\local\programs\python\python37\lib\site-packages\sklearn\metrics\_classification.py:1318: UndefinedMetricWarning: Recall is ill-defined and being set to 0.0 in labels with no true samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
c:\users\ivan\appdata\local\programs\python\python37\lib\site-packages\sklearn\metrics\_classification.py:1318: UndefinedMetricWarning: Recall is ill-defined and being set to 0.0 in labels with no true samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
c:\users\ivan\appdata\local\programs\python\python37\lib\site-packages\sklearn\utils\validation.py:598: FutureWarning: np.matrix usage is deprecated in 1.0 and will raise

...

c:\users\ivan\appdata\local\programs\python\python37\lib\site-packages\sklearn\metrics\_ranking.py:864: RuntimeWarning: invalid value encountered in true_divide
  recall = tps / tps[-1]
c:\users\ivan\appdata\local\programs\python\python37\lib\site-packages\sklearn\metrics\_classification.py:1318: UndefinedMetricWarning: Recall is ill-defined and being set to 0.0 in labels with no true samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
c:\users\ivan\appdata\local\programs\python\python37\lib\site-packages\sklearn\metrics\_classification.py:1318: UndefinedMetricWarning: Recall is ill-defined and being set to 0.0 in labels with no true samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
c:\users\ivan\appdata\local\programs\python\python37\lib\site-packages\sklearn\utils\validation.py:598: FutureWarning: np.matrix usage is deprecated in 1.0 and will raise

10us 147


c:\users\ivan\appdata\local\programs\python\python37\lib\site-packages\sklearn\metrics\_ranking.py:864: RuntimeWarning: invalid value encountered in true_divide
  recall = tps / tps[-1]
c:\users\ivan\appdata\local\programs\python\python37\lib\site-packages\sklearn\metrics\_classification.py:1318: UndefinedMetricWarning: Recall is ill-defined and being set to 0.0 in labels with no true samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
c:\users\ivan\appdata\local\programs\python\python37\lib\site-packages\sklearn\metrics\_classification.py:1318: UndefinedMetricWarning: Recall is ill-defined and being set to 0.0 in labels with no true samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
c:\users\ivan\appdata\local\programs\python\python37\lib\site-packages\sklearn\utils\validation.py:598: FutureWarning: np.matrix usage is deprecated in 1.0 and will raise

10us 148


c:\users\ivan\appdata\local\programs\python\python37\lib\site-packages\sklearn\metrics\_ranking.py:864: RuntimeWarning: invalid value encountered in true_divide
  recall = tps / tps[-1]
c:\users\ivan\appdata\local\programs\python\python37\lib\site-packages\sklearn\metrics\_classification.py:1318: UndefinedMetricWarning: Recall is ill-defined and being set to 0.0 in labels with no true samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
c:\users\ivan\appdata\local\programs\python\python37\lib\site-packages\sklearn\metrics\_classification.py:1318: UndefinedMetricWarning: Recall is ill-defined and being set to 0.0 in labels with no true samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


In [23]:
mean_rmse_score = np.mean(playlist_rmse_scores)    
mean_mae_score = np.mean(playlist_mae_scores)
mean_map_score = np.mean(playlist_map_scores)
mean_ndcg_score = np.mean(playlist_ndcg_scores)


mean_precision_score_cv_micro = np.mean(precision_results_cv_micro)
mean_precision_score_cv_macro = np.mean(precision_results_cv_macro)
mean_precision_score_cv_weighted = np.mean(precision_results_cv_weighted)

mean_recall_score_cv_micro = np.mean(recall_results_cv_micro)
mean_recall_score_cv_macro = np.mean(recall_results_cv_macro)
mean_recall_score_cv_weighted = np.mean(recall_results_cv_weighted)

f1_score_cv_micro=(2*mean_precision_score_cv_micro*mean_recall_score_cv_micro)/(mean_precision_score_cv_micro+mean_recall_score_cv_micro)
f1_score_cv_macro=(2*mean_precision_score_cv_macro*mean_recall_score_cv_macro)/(mean_precision_score_cv_macro+mean_recall_score_cv_macro)
f1_score_cv_weighted=(2*mean_precision_score_cv_weighted*mean_recall_score_cv_weighted)/(mean_precision_score_cv_weighted+mean_recall_score_cv_weighted)


print('RMSE SCORE:   ',mean_rmse_score)
print('MAE SCORE:   ',mean_mae_score)
print('MAP SCORE:   ',mean_map_score)
print('NDCG SCORE:   ',mean_ndcg_score)

print("CV SKLEARN.METRICS PRECISION:    ", mean_precision_score_cv_micro, "|", mean_precision_score_cv_macro, "|",
                                             mean_precision_score_cv_weighted)
print("CV SKLEARN.METRICS RECALL:    ", mean_recall_score_cv_micro, "|", mean_recall_score_cv_macro, "|",
                                             mean_recall_score_cv_weighted)

print("CV SKLEARN.METRICS F1:    ", f1_score_cv_micro, "|", f1_score_cv_macro, "|",
                                             f1_score_cv_weighted)

RMSE SCORE:    0.10725884931754426
MAE SCORE:    0.07707832495983859
MAP SCORE:    nan
NDCG SCORE:    0.0
CV SKLEARN.METRICS PRECISION:     0.8106091759908707 | 0.45 | 0.9
CV SKLEARN.METRICS RECALL:     0.8106091759908707 | 0.40530458799543534 | 0.8106091759908707
CV SKLEARN.METRICS F1:     0.8106091759908707 | 0.4264844761920517 | 0.8529689523841034


In [25]:
print("NOVELTY-CATALOG COVERAGE-PREDICTION COVERAGE-PERSONALIZATION-NOVELTY-CATALOG COVERAGE-PREDICTION COVERAGE-PERSONALIZATION")

NOVELTY-CATALOG COVERAGE-PREDICTION COVERAGE-PERSONALIZATION-NOVELTY-CATALOG COVERAGE-PREDICTION COVERAGE-PERSONALIZATION


In [26]:
#EVALUACIJA NOVELTY
def novelty(predicted: dict, pop: dict, u: int, n: int) -> (float, list):
    
    mean_self_information = []
    k = 0
    for pred25 in predicted: 
        self_information = 0
        k += 1
        for track_index in predicted[pred25].index:
            self_information += np.sum(-np.log2(pop[track_index]/u))
        mean_self_information.append(self_information/n)
    novelty = sum(mean_self_information)/k
    return novelty, mean_self_information
    
test_users = len(playlist_tracks_df_dict) # Total unique users from notebook demo
test_recs_per_user = 100
# WHEN metrics.novelty is run
novelty_score, _ = novelty(
            predicted = preds_dict_rec_100,#predicted data
            pop = items_count_dict, #test data
            u = test_users, #broj korisnika
            n = test_recs_per_user #broj testnih recorda
)

print("NOVELTY SCORE:   ",novelty_score)

NOVELTY SCORE:    0.09305469214862779


In [27]:
#EVALUACIJA CATALOG COVERAGE
def catalog_coverage(catalog: dict) -> float:
    sum_non_zero=0
    for elem in catalog:
        if(catalog[elem]!=0):
            sum_non_zero+=1
    catalog_coverage = round(sum_non_zero/(len(catalog)*1.0),2)
    return catalog_coverage
catalog_coverage = catalog_coverage(
    catalog = items_count_dict, #test data
)

print("CATALOG COVERAGE SCORE:   ",catalog_coverage)


CATALOG COVERAGE SCORE:    0.01


In [29]:
def personalization(predicted: List[list]) -> float:

    def make_rec_matrix(predicted: List[list]) -> sp.csr_matrix:
        df = pd.DataFrame(data=predicted).reset_index().melt(
            id_vars='index', value_name='item',
        )
        df = df[['index', 'item']].pivot(index='index', columns='item', values='item')
        df = pd.notna(df)*1
        rec_matrix = sp.csr_matrix(df.values)
        return rec_matrix

    #create matrix for recommendations
    predicted = np.array(predicted)
    rec_matrix_sparse = make_rec_matrix(predicted)

    #calculate similarity for every user's recommendation list
    similarity = cosine_similarity(X=rec_matrix_sparse, dense_output=False)

    #calculate average similarity
    dim = similarity.shape[0]
    personalization = (similarity.sum() - dim) / (dim * (dim - 1))
    return 1-personalization

pred100_list = [preds_dict_rec_100[playlist].index.tolist() for playlist in preds_dict_rec_100]

# WHEN metrics.personalization is run
personalization_score =personalization(
    predicted=pred100_list)

# THEN the personalization score should be within the expected value
print("PERSONALIZATION SCORE:    ",personalization_score)

PERSONALIZATION SCORE:     0.05514971957925152


In [30]:
def jaccard_similarity(dframe, average_100th):
    index_rating = dframe[dframe['isin']==0].index
    index_pred = dframe[dframe['pred']>average_100th].index
    intersection_size = len(index_rating[index_rating.isin(index_pred)])
    union_size = len(index_rating.union(index_pred))
    return intersection_size / union_size if union_size != 0 else 0.0

def inter_list_diversity(dframe, average_100th):
    length_frame = len(dframe)
    pairwise_similarities = []    
    similarity = jaccard_similarity(dframe, average_100th)    
    return similarity

diversity_score_list=[]
for user in tqdm(content_model_recommendations_dict):
    diversity_score_list.append(inter_list_diversity(content_model_recommendations_dict[user], average_100th))
print("Inter-List Diversity Score:", np.mean(diversity_score_list))

100%|███████████████████████████████████████████████████████████████████████████████| 149/149 [00:00<00:00, 152.78it/s]

Inter-List Diversity Score: 0.015247313033717852
